In [1]:
import numpy as np
import pandas as pd
import os
import torch
from PIL import Image
from collections import defaultdict
from collections import Counter
from tqdm import tqdm, tqdm_notebook
from matplotlib import colors, pyplot as plt

In [2]:
from metrics import Metrics
from custom_dataset import CustomDataset

In [2]:
image_folder_val = '/kaggle/input/military-equipment/Dataset military equipment/images_val'
image_folder_train = '/kaggle/input/military-equipment/Dataset military equipment/images_train'
labels_folder_val = '/kaggle/input/military-equipment/Dataset military equipment/labels_val'
labels_folder_train = '/kaggle/input/military-equipment/Dataset military equipment/labels_train'

In [3]:
RESCALE_SIZE = 320
DATA_MODES = ['train', 'val', 'test']

# New Preprocessing

In [4]:
def custom_transform(img, rescale_size):
    image = img.resize((rescale_size, rescale_size), resample=Image.BILINEAR)
    return image

In [5]:
def crop_and_resize(img, bbox, rescale_size):
    # bbox = (center_x, center_y, width, height) in YOLO format
    img_width, img_height = img.size
    center_x, center_y, bbox_width, bbox_height = bbox

    # Convert YOLO bbox format (relative) to absolute pixel coordinates
    center_x = int(center_x * img_width)
    center_y = int(center_y * img_height)
    bbox_width = int(bbox_width * img_width)
    bbox_height = int(bbox_height * img_height)

    # Compute the bounding box corners
    left = max(0, center_x - bbox_width // 2)
    top = max(0, center_y - bbox_height // 2)
    right = min(img_width, center_x + bbox_width // 2)
    bottom = min(img_height, center_y + bbox_height // 2)

    # Crop and resize the image
    cropped_img = img.crop((left, top, right, bottom))
    resized_img = custom_transform(cropped_img, rescale_size)
    return resized_img

In [6]:
# Initialize lists
train_images = []
train_labels = []

# Process each image and its corresponding label file
for filename in tqdm(os.listdir(image_folder_train)):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # File paths
        file_path = os.path.join(image_folder_train, filename)
        label_file = os.path.join(labels_folder_train, filename.split('.')[0] + ".txt")
        
        try:
            # Open the image
            img = Image.open(file_path)

            # Check if the corresponding label file exists
            if not os.path.exists(label_file):
                print(f"Label file missing for {filename}, skipping.")
                continue
            
            # Read the label file
            with open(label_file, 'r') as file:
                lines = file.readlines()

            # If no bounding boxes are found, skip the image
            if len(lines) == 0:
                print(f"No bounding boxes found in {label_file}, skipping.")
                continue

            # Process each bounding box
            for line in lines:
                # Parse the YOLO format: class_id center_x center_y width height
                parts = line.strip().split()
                class_label = int(parts[0])
                bbox = list(map(float, parts[1:]))

                # If the image has multiple objects, crop it
                if len(lines) > 1:
                    cropped_img = crop_and_resize(img, bbox, RESCALE_SIZE)
                    train_images.append(cropped_img)
                    train_labels.append(class_label)
                else:
                    # If only one object, resize the whole image
                    resized_img = custom_transform(img, RESCALE_SIZE)
                    train_images.append(resized_img)
                    train_labels.append(class_label)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

100%|██████████| 11768/11768 [04:20<00:00, 45.23it/s]


In [7]:
# Initialize lists
val_images = []
val_labels = []

# Process each image and its corresponding label file
for filename in tqdm(os.listdir(image_folder_val)):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        # File paths
        file_path = os.path.join(image_folder_val, filename)
        label_file = os.path.join(labels_folder_val, filename.split('.')[0] + ".txt")
        
        try:
            # Open the image
            img = Image.open(file_path)

            # Check if the corresponding label file exists
            if not os.path.exists(label_file):
                print(f"Label file missing for {filename}, skipping.")
                continue
            
            # Read the label file
            with open(label_file, 'r') as file:
                lines = file.readlines()

            # If no bounding boxes are found, skip the image
            if len(lines) == 0:
                print(f"No bounding boxes found in {label_file}, skipping.")
                continue

            # Process each bounding box
            for line in lines:
                # Parse the YOLO format: class_id center_x center_y width height
                parts = line.strip().split()
                class_label = int(parts[0])
                bbox = list(map(float, parts[1:]))

                # If the image has multiple objects, crop it
                if len(lines) > 1:
                    cropped_img = crop_and_resize(img, bbox, RESCALE_SIZE)
                    val_images.append(cropped_img)
                    val_labels.append(class_label)
                else:
                    # If only one object, resize the whole image
                    resized_img = custom_transform(img, RESCALE_SIZE)
                    val_images.append(resized_img)
                    val_labels.append(class_label)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

100%|██████████| 1680/1680 [00:37<00:00, 45.16it/s]


In [9]:
class_counts = Counter(train_labels)
print("\nClass Label Counts:")
for cls, count in class_counts.items():
    print(f"Class {cls}: {count} instances")


Class Label Counts:
Class 4: 1911 instances
Class 9: 2067 instances
Class 0: 5376 instances
Class 3: 1845 instances
Class 2: 7367 instances
Class 10: 4958 instances
Class 1: 1949 instances
Class 6: 891 instances
Class 5: 826 instances
Class 7: 2538 instances
Class 8: 1328 instances


In [10]:
# Total number of instances
total_instances = sum(class_counts.values())

# Calculate class weights (inverse frequency)
class_weights = {cls: total_instances / count for cls, count in class_counts.items()}

# Normalize weights (optional)
max_weight = max(class_weights.values())
class_weights = {cls: weight / max_weight for cls, weight in class_weights.items()}

# Convert class weights to a PyTorch tensor
num_classes = len(class_counts)
weight_tensor = torch.zeros(num_classes)
for cls, weight in class_weights.items():
    weight_tensor[cls] = weight

print("Class Weights Tensor:", weight_tensor)

Class Weights Tensor: tensor([0.1536, 0.4238, 0.1121, 0.4477, 0.4322, 1.0000, 0.9270, 0.3255, 0.6220,
        0.3996, 0.1666])


# Image shapes

In [11]:
channel_sum = np.zeros(3)          # Sum of pixel values per channel (R, G, B)
channel_squared_sum = np.zeros(3) # Sum of squared pixel values per channel
num_pixels = 0                    # Total number of pixels across all images

# Process images in batches
batch_size = 100
for i in tqdm(range(0, len(train_images), batch_size)):
    # Get a batch of images
    batch_images = train_images[i:i + batch_size]
    
    # Process each image in the batch
    for img in batch_images:
        # Convert image to NumPy array and normalize to [0, 1]
        img_array = np.array(img) / 255.0  # Shape: (320, 320, 3)

        # Accumulate sums and squared sums
        channel_sum += img_array.sum(axis=(0, 1))  # Sum across height and width
        channel_squared_sum += (img_array ** 2).sum(axis=(0, 1))  # Squared sum
        num_pixels += img_array.shape[0] * img_array.shape[1]  # Total number of pixels in the image

# Calculate mean and standard deviation
channel_mean = channel_sum / num_pixels
channel_std = np.sqrt(channel_squared_sum / num_pixels - channel_mean ** 2)

100%|██████████| 311/311 [02:13<00:00,  2.32it/s]


In [12]:
print("Channel Means: ", channel_mean)
print("Channel Standard Deviations: ", channel_std)

Channel Means:  [0.44706843 0.45275457 0.43323347]
Channel Standard Deviations:  [0.1888887  0.18831457 0.19633765]


In [13]:
# Dictionary to store image shapes and their counts
shape_counts = defaultdict(int)

# Loop through each file in the directory
for filename in os.listdir(image_folder_train):
    if filename.endswith(".png") or filename.endswith(".jpg"):
        file_path = os.path.join(image_folder_train, filename)
        try:
            # Open and load the image using Pillow
            img = Image.open(file_path)
            
            # Get the shape of the image (width, height)
            shape = img.size  # (width, height)
            
            # Increment the count for this shape
            shape_counts[shape] += 1
        except Exception as e:
            print(f"Error loading {filename}: {e}")

# Print grouped shapes with the number of instances
print("Image Shapes and Counts:")
for shape, count in shape_counts.items():
    print(f"Shape {shape}: {count} instances")

# Total number of images processed
print(f"Total unique shapes: {len(shape_counts)}")

Image Shapes and Counts:
Shape (1280, 720): 6446 instances
Shape (1920, 1080): 1607 instances
Shape (854, 480): 1472 instances
Shape (512, 288): 786 instances
Shape (1280, 702): 86 instances
Shape (912, 608): 319 instances
Shape (576, 320): 550 instances
Shape (720, 480): 302 instances
Shape (558, 360): 89 instances
Shape (1280, 704): 70 instances
Shape (888, 490): 40 instances
Shape (320, 352): 1 instances
Total unique shapes: 12


# Labels

In [14]:
print("\nClass Weights and Class Names:")
for cls, weight in class_weights.items():
    print(f"Class {cls}: Weight = {weight:.4f}")


Class Weights and Class Names:
Class 4: Weight = 0.4322
Class 9: Weight = 0.3996
Class 0: Weight = 0.1536
Class 3: Weight = 0.4477
Class 2: Weight = 0.1121
Class 10: Weight = 0.1666
Class 1: Weight = 0.4238
Class 6: Weight = 0.9270
Class 5: Weight = 1.0000
Class 7: Weight = 0.3255
Class 8: Weight = 0.6220


# Transorm

In [15]:
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [16]:
MEAN = [0.5097, 0.524, 0.5099]
STD = [0.212, 0.212, 0.237]

In [17]:
transform_v1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomRotation(degrees=25),
    transforms.RandomPerspective(distortion_scale=0.6, p=0.25),
    transforms.Normalize(MEAN, STD)
])

# DataSet

In [18]:
class SimpsonsDataset(Dataset):
    def __init__(self, images, labels, mode, transform):
        super().__init__()
        # режим работы
        self.mode = mode
        self.transform = transform
        self.labels = labels
        self.images = images

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.images)

    def __len__(self):
        return self.len_

    def __getitem__(self, index):
        x = self.transform(self.images[index])
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            return x, torch.tensor(label, dtype=torch.long)

# Model

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn import init
import math

In [20]:
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction_ratio, bias=False),
            nn.ReLU(),
            nn.Linear(in_channels // reduction_ratio, in_channels, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
        self.init_weights()

    def init_weights(self):
        for layer in self.fc:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)

    def forward(self, x):
        b, c, h, w = x.size()
        avg_out = self.fc(self.avg_pool(x).view(b, c))
        max_out = self.fc(self.max_pool(x).view(b, c))
        out = avg_out + max_out
        return self.sigmoid(out).view(b, c, 1, 1) * x

class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        concat = torch.cat([avg_out, max_out], dim=1) # -1?
        attention = self.sigmoid(self.conv(concat))
        return attention * x

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction_ratio=16):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction_ratio)
        self.spatial_attention = SpatialAttention()

    def forward(self, x):
        x = self.channel_attention(x)
        x = self.spatial_attention(x)
        return x

In [21]:
class ResNet18WithCBAM(nn.Module):
    def __init__(self, num_classes, pretrained=True):
        super(ResNet18WithCBAM, self).__init__()
        
        self.resnet = models.resnet18(pretrained=pretrained)
        self.layer0 = nn.Sequential(
            self.resnet.conv1,
            self.resnet.bn1,
            self.resnet.relu,
            self.resnet.maxpool
        )
        self.layer1 = self.resnet.layer1
        self.layer2 = self.resnet.layer2
        self.layer3 = self.resnet.layer3
        self.layer4 = self.resnet.layer4
        self.avgpool = self.resnet.avgpool
        in_features = self.resnet.fc.in_features  # Get the number of input features for the original FC layer
        self.fc = nn.Linear(in_features, num_classes)  # New FC layer
        
        self.module1 = CBAM(64)
        self.module2 = CBAM(128)
        self.module3 = CBAM(256)
        self.module4 = CBAM(512)
        nn.init.xavier_uniform_(self.fc.weight)


    def get_info(self):
        return 'ResNet18', 'CBAM'


    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.module1(x)
        x = self.layer2(x)
        x = self.module2(x)
        x = self.layer3(x)
        x = self.module3(x)
        x = self.layer4(x)
        x = self.module4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [22]:
model = ResNet18WithCBAM(num_classes=11, pretrained=True)
input_tensor = torch.randn(1, 3, 224, 224)  # Example input
output = model(input_tensor)
print("Output shape:", output.shape)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 175MB/s]


Output shape: torch.Size([1, 11])


In [23]:
class SpatialGroupEnhance(nn.Module):
    def __init__(self, groups = 64):
        super(SpatialGroupEnhance, self).__init__()
        self.groups = groups
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.weight = Parameter(torch.zeros(1, groups, 1, 1))
        self.bias = Parameter(torch.ones(1, groups, 1, 1))
        self.sig = nn.Sigmoid()

    def forward(self, x): # (b, c, h, w)
        b, c, h, w = x.size()
        x = x.view(b * self.groups, -1, h, w) 
        xn = x * self.avg_pool(x)
        xn = xn.sum(dim=1, keepdim=True)
        t = xn.view(b * self.groups, -1)
        t = t - t.mean(dim=1, keepdim=True)
        std = t.std(dim=1, keepdim=True) + 1e-5
        t = t / std
        t = t.view(b, self.groups, h, w)
        t = t * self.weight + self.bias
        t = t.view(b * self.groups, 1, h, w)
        x = x * self.sig(t)
        x = x.view(b, c, h, w)
        return x

In [24]:
class ResNet18WithSGE(nn.Module):
    def __init__(self, num_classes, groups, pretrained=True):
        super(ResNet18WithSGE, self).__init__()
        
        self.resnet = models.resnet18(pretrained=pretrained)
        self.layer0 = nn.Sequential(
            self.resnet.conv1,
            self.resnet.bn1,
            self.resnet.relu,
            self.resnet.maxpool
        )
        self.layer1 = self.resnet.layer1
        self.layer2 = self.resnet.layer2
        self.layer3 = self.resnet.layer3
        self.layer4 = self.resnet.layer4
        self.avgpool = self.resnet.avgpool
        in_features = self.resnet.fc.in_features  # Get the number of input features for the original FC layer
        self.fc = nn.Linear(in_features, num_classes)  # New FC layer
        
        self.module1 = SpatialGroupEnhance(groups[0])
        self.module2 = SpatialGroupEnhance(groups[1])
        self.module3 = SpatialGroupEnhance(groups[2])
        self.module4 = SpatialGroupEnhance(groups[3])
        nn.init.xavier_uniform_(self.fc.weight)


    def get_info(self):
        return 'ResNet18', 'SGE'
    

    def forward(self, x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.module1(x)
        x = self.layer2(x)
        x = self.module2(x)
        x = self.layer3(x)
        x = self.module3(x)
        x = self.layer4(x)
        x = self.module4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [25]:
model2 = ResNet18WithSGE(num_classes=11, groups=[2, 4, 4, 4], pretrained=True)

In [26]:
pretrained_params = []
cbam_params = []

for name, param in model2.named_parameters():
    if 'module' in name or 'fc' in name:
        cbam_params.append(param)
    else:
        pretrained_params.append(param)

In [27]:
# Optimizers with different learning rates
optimizer = optim.AdamW([
    {'params': pretrained_params, 'lr': 5e-5, 'weight_decay': 1e-4},  # Smaller LR for pretrained layers
    {'params': cbam_params, 'lr': 5e-4, 'weight_decay': 1e-3}        # Larger LR for CBAM layers
])

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [29]:
criterion = torch.nn.CrossEntropyLoss(weight=weight_tensor.to(device))

In [30]:
model2 = model2.to(device)

# Resnet

In [31]:
resnet18 = models.resnet18(pretrained=True)

In [32]:
num_classes = 11
in_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(in_features, num_classes)

In [33]:
pretrained_params = []
last_layer_params = []

for name, param in resnet18.named_parameters():
    if name.startswith("fc"):  # fc refers to the last layer
        last_layer_params.append(param)
    else:
        pretrained_params.append(param)

In [34]:
optimizer = optim.AdamW([
    {'params': pretrained_params, 'lr': 5e-5, 'weight_decay': 1e-4},  # Smaller LR for pretrained layers
    {'params': last_layer_params, 'lr': 5e-4, 'weight_decay': 1e-3}  # Larger LR for the last layer
])

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
criterion = torch.nn.CrossEntropyLoss(weight=weight_tensor.to(device))

In [37]:
resnet18 = resnet18.to(device)

# Metrics

In [38]:
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score

In [39]:
class Metrics():
    def __init__(self):
        self.all_actual = np.array([])
        self.all_predicted = np.array([])

    def batch_step(self, actualv, predictedv):
        self.all_actual = np.concatenate([self.all_actual, actualv.numpy(force=True)])
        self.all_predicted = np.concatenate([self.all_predicted, predictedv.numpy(force=True)])

    def get_metrics(self):
        recall = recall_score(self.all_actual, self.all_predicted, average='weighted', zero_division=0)
        precision = precision_score(self.all_actual, self.all_predicted, average='weighted', zero_division=0)
        f1 = f1_score(self.all_actual, self.all_predicted, average='weighted', zero_division=0)
        accuracy = accuracy_score(self.all_actual, self.all_predicted)
        #roc_auc = roc_auc_score(self.all_actual, self.all_predicted, average='weighted', multi_class='ovo')
        return recall, precision, f1, accuracy

# Train function

In [40]:
import time
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

In [41]:
BATCH_SIZE = 64

In [42]:
train_dataset = SimpsonsDataset(train_images, train_labels, 'train', transform_v1)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
val_dataset = SimpsonsDataset(val_images, val_labels, 'val', transform_v1)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=4)

In [43]:
def train_step(model, loss_fn, opt, loader, batch_size):
    loss_per_batches = 0
    elapsed = 0
    start_epoch2 = time.time()
    for i, data in tqdm(enumerate(loader), total=len(train_labels)//batch_size):

        start_epoch = time.time()
        features, labels = data
        features, labels = features.to(device), labels.to(device)
        opt.zero_grad()

        y_pred = model(features)
        loss = loss_fn(y_pred, labels)
        loss.backward()

        opt.step()

        loss_per_batches += loss
        end_epoch = time.time()
        elapsed += (end_epoch - start_epoch)

    print("train = " + str(elapsed))
    print("train + load = " + str(time.time() - start_epoch2))
    return loss_per_batches/(i+1)

In [44]:
loss_val = list()
loss_train = list()
acc_val = list()

In [45]:
directory_path = '/kaggle/working/model_svs/'

In [46]:
if not os.path.exists(directory_path):
    # Create the directory
    os.makedirs(directory_path)
    print(f"Directory created at: {directory_path}")
else:
    print(f"Directory already exists at: {directory_path}")

Directory created at: /kaggle/working/model_svs/


In [47]:
def train(model, loss_fn, opt, train_loader, val_loader, batch_size, save_treshold=10, epochs=50, model_name='model_name'):

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    model_type, module = model.get_info()
    writer = SummaryWriter('/kaggle/working/runs/' + model_name + '{}_{}_{}'.format(model_type, module, timestamp))
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(opt, epochs)
    

    for epoch in range(epochs):
        start_epoch = time.time()
        metrics_valid = Metrics()
        print('EPOCH {}:'.format(epoch + 1))

        model.train()
        avg_loss = train_step(model, loss_fn, opt, train_loader, batch_size)
        model.eval()

        vloss = 0
        counter = 0
        with torch.inference_mode():
            for i, vdata in enumerate(val_loader):
                vfeatures, vlabels = vdata
                vfeatures, vlabels = vfeatures.to(device), vlabels.to(device)

                y_pred = model(vfeatures)
                bloss = loss_fn(y_pred, vlabels)
                vloss += bloss
                y_pred = torch.argmax(y_pred, 1)
                metrics_valid.batch_step(vlabels, y_pred)
                counter = i

        avg_vloss = vloss / (counter + 1)

        scheduler.step()

        valrecall, valprecision, valf1, valacc = metrics_valid.get_metrics()
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
        print('Accuracy valid {}'.format(valacc))
        print('Recall valid {}'.format(valrecall))
        print('Precision valid {}'.format(valprecision))
        print('Val F1->{}'.format(valf1))
        #print('roc auc valid {}'.format(roc_auc))

        loss_train.append(avg_loss)
        loss_val.append(avg_vloss)
        acc_val.append(valacc)
        writer.add_scalars('Training vs. Validation Loss',
                    { 'Training': avg_loss, 'Validation': avg_vloss },
                    epoch + 1)
        writer.add_scalars('Validation Metrics',
                    { 'Validation Recall': valrecall, 'Validation Precision': valprecision, 'Validation F1': valf1
                    }, epoch + 1)
        writer.add_scalars('Validation Accuracy',
                    { 'Validation Accuracy': valacc
                    }, epoch + 1)

        if (epoch + 1) % save_treshold == 0:
            model_path = '/kaggle/working/model_svs/' + model_name +'_{}_{}_{}_{}'.format(model_type, module, 
                                                                          timestamp, (epoch + 1))
            torch.save(model.state_dict(), model_path)
        end_epoch = time.time()
        elapsed = end_epoch - start_epoch
        print("Time per epoch {}s".format(elapsed))

In [48]:
train(model2, criterion, optimizer, train_dataloader, val_dataloader, BATCH_SIZE, save_treshold=1, epochs=50, model_name='second_model')

EPOCH 1:


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
486it [02:11,  3.69it/s]

train = 72.71642065048218
train + load = 132.4729835987091


LOSS train 3.0176494121551514 valid 3.0220391750335693
Accuracy valid 0.1742929409059554
Recall valid 0.1742929409059554
Precision valid 0.11014275311152587
Val F1->0.08897493428777403
Time per epoch 151.12501001358032s
EPOCH 2:


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
486it [02:09,  3.74it/s]

...

train = 71.19407820701599
train + load = 148.13710045814514


LOSS train 3.019299268722534 valid nan
Accuracy valid 0.17314325132214303
Recall valid 0.17314325132214303
Precision valid 0.02997858547840278
Val F1->0.051108141217394626
Time per epoch 170.41105771064758s
EPOCH 50:


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
486it [02:41,  3.01it/s]

train = 76.99123764038086
train + load = 162.19154167175293


LOSS train 3.0180118083953857 valid nan
Accuracy valid 0.17314325132214303
Recall valid 0.17314325132214303
Precision valid 0.02997858547840278
Val F1->0.051108141217394626
Time per epoch 185.27664804458618s


# Test

In [49]:
torch.cuda.empty_cache()

# Optionally, you can force garbage collection as well
import gc
gc.collect()

0

In [50]:
9999

9999

In [51]:
import shutil

In [52]:
# Paths for the directories
output_dir = "/kaggle/working/"
zip_path = "/kaggle/working/output2.zip"

# Create a zip file
shutil.make_archive("/kaggle/working/output", 'zip', output_dir)

'/kaggle/working/output.zip'

In [53]:
test_set = SimpsonsDataset(train_images, train_labels, 'train', transform_v1)
test_loader = DataLoader(train_dataset, batch_size=128, num_workers=4, shuffle=True)

In [54]:
temp = 0
for i, data in tqdm(enumerate(test_loader), total=len(train_labels)//128):
    temp += 1

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
243it [02:23,  1.69it/s]                         


In [55]:
train_dataset = SimpsonsDataset(train_images[:100], train_labels[:100], 'train', transform_v1)
train_dataloader = DataLoader(train_dataset, batch_size=64, num_workers=0, shuffle=True)
val_dataset = SimpsonsDataset(val_images[:100], val_labels[:100], 'val', transform_v1)
val_dataloader = DataLoader(val_dataset, batch_size=64, num_workers=0)

In [56]:
print(124)

124
